In [1]:
# pre-requisites
# creating dataset - 500 images bnw and 500 color images
# create a labels.txt file and keep it with test and train directory at same level
# partion it into train and test images
# building tensorflow 'inception' inside models directory
# building 'slim' a high level API inside models directory

In [2]:
# creating custom TFRF record file by running - 
# bazel-bin/inception/build_image_data \
#   --train_directory="/home/siftr/BNW/dataset/train/" \
#   --validation_directory="/home/siftr/BNW/dataset/test/" \
#   --output_directory="/home/siftr/BNW/TFRF/" \
#   --labels_file="/home/siftr/BNW/dataset/labels.txt" \
#   --train_shards=128 \
#   --validation_shards=24 \
#   --num_threads=8
# look here if stuck - https://github.com/tensorflow/models/tree/master/inception#how-to-construct-a-new-dataset-for-retraining

In [3]:
# create custom dataset factory to read data insidel slim/datasets/
# cp flowers.py custom_data.py
# edit custom_data.py for number of validation/train data you have
# register this custom_data in dataset_factory.py

In [4]:
import tensorflow as tf
from tensorflow.models.slim.datasets import custom_data, dataset_factory

In [5]:
slim = tf.contrib.slim

In [6]:
def preprocessing(image, size):
    image = tf.to_float(image)
    image = tf.image.resize_image_with_crop_or_pad(image, size, size)
    return image

In [7]:
def inference(images, num_classes=2, is_training=True, prediction_fn=slim.softmax, scope='bnw'):
    with slim.arg_scope([slim.conv2d, slim.fully_connected], activation_fn=tf.nn.relu):
        with tf.variable_scope(scope, 'bnw', [images, num_classes]):
            net = slim.conv2d(images, 64, [5, 5], scope='conv1')
            print("conv1 --> %s"%net.get_shape())
            net = slim.max_pool2d(net, [2, 2], stride=2, scope='pool1')
            print("pool1 --> %s"%net.get_shape())
            net = slim.conv2d(images, 64, [5, 5], scope='conv2')
            print("conv2 --> %s"%net.get_shape())
            net = slim.max_pool2d(net, [2, 2], stride=2, scope='pool2')
            print("pool2 --> %s"%net.get_shape())
            net = slim.fully_connected(net, 192, scope='fc1')
            print("fc1 --> %s"%net.get_shape())
            net = slim.fully_connected(net, 2, scope='fc2')
            print("fc2 --> %s"%net.get_shape())
            net = slim.max_pool2d(net, [16, 16], scope='pool3')
            print("pool3 --> %s"%net.get_shape())
            logits = tf.squeeze(net, [1, 2], name='logits')
            print("logits --> %s"%net.get_shape())
    return logits

In [8]:
with tf.Graph().as_default():
    global_step = slim.create_global_step()
    dataset = dataset_factory.get_dataset('custom_data', 'train', '/home/siftr/BNW/TFRF/')
    provider = slim.dataset_data_provider.DatasetDataProvider(dataset, num_readers=2,
                        common_queue_capacity=20 * 16, common_queue_min=10 * 16)
    [image, label] = provider.get(['image', 'label'])
    image = preprocessing(image, 32)
    images, labels = tf.train.batch([image, label], batch_size=16, num_threads=2, capacity=5 * 16)
    labels = slim.one_hot_encoding(labels, 2)
    batch_queue = slim.prefetch_queue.prefetch_queue([images, labels], capacity=2 * 1)
    
    predictions = inference(images, is_training=True)
    slim.losses.softmax_cross_entropy(predictions, labels)
    total_loss = slim.losses.get_total_loss()
    
    learning_rate = tf.train.exponential_decay(0.01, global_step, 500, 0.75, staircase=True, name='exponential_decay_learning_rate')
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_tensor = slim.learning.create_train_op(total_loss, optimizer)
    slim.learning.train(
        train_tensor,
        "/home/siftr/BNW/demologs/",
        number_of_steps=1,
        save_summaries_secs=120,
    )

conv1 --> (16, 32, 32, 64)
pool1 --> (16, 16, 16, 64)
conv2 --> (16, 32, 32, 64)
pool2 --> (16, 16, 16, 64)
fc1 --> (16, 16, 16, 192)
fc2 --> (16, 16, 16, 2)
pool3 --> (16, 1, 1, 2)
logits --> (16, 1, 1, 2)
Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
Instructions for updating:
Use tf.losses.get_total_loss instead.
Instructions for updating:
Use tf.losses.get_losses instead.
Instructions for updating:
Use tf.losses.get_regularization_losses instead.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global step 1: loss = 4.9408 (0.40 sec/step)
INFO:tensorflow:Stopping Training.
INFO:tensorflow:Finished training! Saving model to disk.
